In [1]:
## Hipótese e Inferência

In [2]:
from typing import Tuple
import math

import import_ipynb

In [3]:
def normal_approximation_to_binomial(n: int, p: float) -> Tuple[float, float]:
    """Retorna mu e sigma correspondentes Binomial(n, p)"""
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

In [13]:
from probability import normal_cdf

# o normal cdf é a probabilidade de a variável estar abaixo de um limite
normal_probability_below = normal_cdf

# está acima do limite se não está abaixo do limite
def normal_probability_above(lo: float,
                             mu: float = 0,
                             sigma: float = 1) -> float:
    """A probabilidade de que um número N(mu, sigma) seja maior do que lo."""
    return 1 - normal_cdf(lo, mu, sigma)

# está entre se é menor do que hi, mas não menor do que low
def normal_probability_between(lo: float,
                               hi:float,
                               mu: float = 0,
                               sigma: float = 1) -> float:
    """A probabilidade de que um número N(mu, sigma) esteja entre low e high"""
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# está fora se não esta dentro
def normal_probabilityhi(lo: float,
                         hi:float,
                         mu: float = 0,
                         sigma: float = 1) -> float:
    """A probabilidade de que um número N(mu, sigma) não esteja entre low e high"""
    
    return 1 - normal_probability_between(lo, hi, mu, sigma)

In [14]:
from probability import inverse_normal_cdf

def normal_upper_bound(probability: float,
                       mu: float = 0,
                       sigma: float = 1) -> float:
    """Retorna z para qual P(Z <= z) = Probalidade"""
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability: float,
                       mu: float = 0,
                       sigma: float = 1) -> float:
    """retorna z para qual P(Z >= z) = Probalidade"""
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability: float,
                            mu: float = 0,
                            sigma: float = 1) -> float:
    """
    Retorna os limites simétricos (relativos à média) que contem a probabilidade especificada
    """
    tail_probability = (1 - probability) / 2

    # o limite superior deve estar abaixo de tail_probability
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)

    # o limite superior deve estar abaixo de tail_probability
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)

    return lower_bound, upper_bound

In [15]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)
lower_bound, upper_bound = normal_two_sided_bounds(0.95, mu_0, sigma_0)

lower_bound, upper_bound

(469.01026640487555, 530.9897335951244)

In [17]:
# com a moeda levemente viciada em cara (p=0.55)

# limites de 95% baseados na premissa de que p é 0.5
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

#mu e sigma reais baseados em p = 0.55
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

# um erro tipo 2 ocorre quando falhamos em rejeitar a hipótese nula,
# o que ocorre quando X ainda está no intervalo original
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability

power


0.8865480012953671

In [19]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)
hi

type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability

power

0.9363794803307173

## P-Values

In [24]:
def two_sided_p_values(x: float, mu: float = 0, sigma: float = 1) -> float:
    """
    Qual é a probabilidade de observar um valor pelo menos tão extremo 
    quanto de x (em qualquer direção) se os valores vem de um N(mu, sigma)
    """

    if x >= mu:
        # x é maior que a média, então a coroa é qualquer valor maior que x
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # x é menor que a média, então a coroa é qualquer valor menor que x
        return 2 * normal_probability_below(x, mu, sigma)
    
two_sided_p_values(529.5, mu_0, sigma_0)

0.06207721579598835

In [37]:
import random

extreme_value_count = 0
for _ in range(1000):
    num_heads = sum(1 if random.random() < 0.5 else 0    # Count # of heads
                    for _ in range(1000))                # in 1000 flips,
    if num_heads >= 530 or num_heads <= 470:             # and count how often
        extreme_value_count += 1                         # the # is 'extreme'

# p-value was 0.062 => ~62 extreme values out of 1000
assert 59 < extreme_value_count < 65, f"{extreme_value_count}"
extreme_value_count

64

In [25]:
two_sided_p_values(531.5, mu_0, sigma_0)

0.046345287837786575

In [38]:
upper_p_value = normal_probability_above
lower_p_value = normal_probability_below

upper_p_value(524.5, mu_0, sigma_0)

0.06062885772582072

In [39]:
upper_p_value(526.5, mu_0, sigma_0)

0.04686839508859242

### Intervalo de confiança

In [40]:
math.sqrt(p * (1 - p) / 1000)

NameError: name 'p' is not defined

In [48]:
p_hat = 525 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)

sigma, normal_two_sided_bounds(0.95, mu, sigma)

(0.015791611697353755, (0.4940490278129096, 0.5559509721870904))

In [47]:
p_hat = 540 / 1000

mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)

sigma, normal_two_sided_bounds(0.95, mu, sigma)

(0.015760710643876435, (0.5091095927295919, 0.5708904072704082))

### p-Hacking

In [52]:
from typing import List

def run_experiment() -> List[bool]:
    """Lanca uma moeda honesta 1000 vezes, True=heads, False=tails"""
    return [random.random() < 0.5 for _ in range(1000)]

def reject_fairness(experiment: List[bool]) -> bool:
    """Use os níveis significancia de 5%"""
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

random.seed(0)

experiments = [run_experiment() for _ in range(1000)]
num_rejections = len([experiment
                      for experiment in experiments
                      if reject_fairness(experiment)])

assert  num_rejections == 46

### Executando um teste A/B

In [55]:
def estimated_parameters(N: int, n: int) -> Tuple[float, float]:
    p = n / N
    sigma = math.sqrt(p * (1 - p) / N)
    return p, sigma

In [59]:
def a_b_test_statistic(N_A: int, n_A: int, N_B: int, n_B: int) -> float:
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

z = a_b_test_statistic(1000, 200, 1000, 180)
z, two_sided_p_values(z)


(-1.1403464899034472, 0.254141976542236)

In [60]:
z = a_b_test_statistic(1000, 200, 1000, 150)
z, two_sided_p_values(z)

(-2.948839123097944, 0.003189699706216853)

### Inferência Baysiana

In [61]:
def B(alpha: float, beta: float) -> float:
    """Uma constante normalizadora para a qual a probabilidade total é 1"""
    return math.gamma(alpha) * math.gamma(beta) / math.gama(alpha + beta)

def beta_pdf(x: float, alpha: float, beta: float) -> float:
    if x <= 0 or x >= 1:
        return 0
    return x ** (alpha - 1) - (1 - x) ** (beta - 1) / B(alpha, beta)


NameError: name 'alpha' is not defined